### Use the 100 gb ram instance

In [45]:
import pandas as pd
from tables import *
import csv
import numpy as np
import pickle
import time
from scipy import sparse
from tqdm import tqdm_notebook as tqdm
import sys
csv.field_size_limit(sys.maxsize)

9223372036854775807

In [2]:
path = '/home/User1/data/self_citations/'

In [3]:
articles = {}
with open(path+'article.txt','r') as f:
    reader = csv.reader(f,delimiter='\t')
    next(reader)
    for line in reader:
        articles[int(line[0])] = int(line[2])

In [4]:
with open(path+'dict_cluster_art.p','rb') as f:
    dict_cluster_art = pickle.load(f)
with open(path+'dict_cluster_ID.p','rb') as f:
    dict_cluster_ID = pickle.load(f)

In [5]:
min_year = 1898
max_year = 2019
nb_years = max_year-min_year

In [208]:
authors = list(dict_cluster_ID.keys())
nb_authors = len(authors)
idx = np.arange(nb_authors)
np.random.shuffle(idx)


In [230]:
IDs_network = []
with open(path+'author_network.npy','wb') as f:
    for i in tqdm(range(nb_authors),smoothing=0):
        cur_author = authors[idx[i]]
        arts_author = dict_cluster_ID[cur_author]
        network = None
        for artID in arts_author:
            co_authors_art = set(dict_cluster_art[artID])
            rm_authors = set([cur_author])
            year_art = articles[artID]
            if len(co_authors_art) > 1:
                if network is None:    
                    new_coauthors = co_authors_art.difference(rm_authors)
                    add_array = np.zeros((2,len(new_coauthors)))
                    add_array[0,:] = np.array(list(new_coauthors))
                    add_array[1,:] = year_art
                    network = add_array
                else:
                    old_coauthors = set(network[0,:])
                    rm_authors = rm_authors|old_coauthors
                    new_coauthors = co_authors_art.difference(rm_authors)
                    if len(new_coauthors)>0:
                        add_array = np.zeros((2,len(new_coauthors)))
                        add_array[0,:] = np.array(list(new_coauthors))
                        add_array[1,:] = year_art
                        network = np.concatenate([network,add_array],axis=1)
        if network is not None:
            IDs_network.append(cur_author)
            np.save(f, network)
            #f.write('\n')



/home/User1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


A Jupyter Widget

In [232]:
with open(path+'IDs_network.p','wb') as f:
    pickle.dump(IDs_network,f)

In [239]:
#with open(path+'IDs_network.p','rb') as f:
#    IDs_network = pickle.load(f)
authors_network = {}
count = 0
with open(path+'author_network.npy','rb') as f1:
    while True:
        try:
            authors_network[IDs_network[count]] = np.load(f1)
            count += 1
        except ValueError:
            break
        if count > 50:
            break